In [55]:
import numpy as np
import sqlite3 as sql
import pandas as pd
import seaborn as sns
import string
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Farahi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

# the Objective is to find wether a review is positive or not

In [2]:
con = sql.connect('data/database.sqlite')

In [3]:
data = pd.read_sql_query(
    """
    select * 
    from Reviews
    where Score != 3
    """,
    con
)

In [4]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
def partition(x) -> str:
    if x < 3:
        return "Negative"
    else:
        return "Positive"


In [6]:
data.Score = data["Score"].map(partition)

In [7]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,Negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


# Data Cleaning: Dedublication

In [8]:
display = pd.read_sql_query(
    """
    select *
    from Reviews
    where Score != 3 and UserId = "AR5J8UI46CURR"
    order by ProductId
    """,
    con
)

In [9]:
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [10]:
sorted_data = data.sort_values("ProductId", axis=0, ascending=True)
sorted_data.shape

(525814, 10)

In [11]:
final = sorted_data.drop_duplicates(subset=["UserId", "ProfileName", "Text", "Time"], keep="first")

In [12]:
final.shape

(364173, 10)

In [13]:
#checking how much data we left with it
(final['Id'].size*1.0)/(data['Id'].size*1.0)*100

69.25890143662969

In [14]:
display = pd.read_sql_query(
    """
    select *
    from Reviews
    where Score != 3 and Id = 44737 or Id = 64422
    order by ProductId
    """,
    con
)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [15]:
final = final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]
final.shape

(364171, 10)

In [16]:
final["Score"].value_counts(normalize=True)

Positive    0.843178
Negative    0.156822
Name: Score, dtype: float64

# Bag of Words (BOW)

In [35]:
corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?', 
]

vectorize = CountVectorizer()


In [38]:
X = vectorize.fit_transform(corpus)
X.shape

(4, 9)

In [46]:
print(X.toarray())
vectorize.get_feature_names_out()


[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]


array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third',
       'this'], dtype=object)

In [49]:
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X2 = vectorizer2.fit_transform(corpus)
vectorizer2.get_feature_names_out()

array(['and this', 'document is', 'first document', 'is the', 'is this',
       'second document', 'the first', 'the second', 'the third',
       'third one', 'this document', 'this is', 'this the'], dtype=object)

In [50]:
print(X2.toarray())

[[0 0 1 1 0 0 1 0 0 0 0 1 0]
 [0 1 0 1 0 1 0 1 0 0 1 0 0]
 [1 0 0 1 0 0 0 0 1 1 0 1 0]
 [0 0 1 0 1 0 1 0 0 0 0 0 1]]


In [ ]:
count_vec = CountVectorizer()
final_count = count_vec.fit_transform(final.Text.values)

In [45]:
final_count.shape

(364171, 115281)

# Text Preprocessing: Stemming, stop-word removal and Lemmatization

In [51]:
import re
#  www.pymotw.com/2/re/
i = 0
for sent in final.Text.values:
    if(len(re.findall('<.*?>', sent))):
        print(i)
        print(sent)
        break
    i += 1


6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [67]:
stop = set(stopwords.words('english'))
sno = nltk.stem.SnowballStemmer("english")

def cleanHtml(sent):
    clear = re.compile('<.*?>')
    clean_text = re.sub(clear, ' ', sent)
    return clean_text

def cleanPunc(sent):
    clean = re.sub(r'[?|!|\'|"|#]', r'', sent)
    clean = re.sub(r'[.|,|)|(|\|/]', r'', sent)
    return clean

In [69]:
print(stop)
print("_________________________________________________________________")
print(sno.stem("tasty"))

{'the', 'or', 'you', 'further', 'myself', 've', "isn't", "mustn't", 'those', 'are', "weren't", 'that', 'hadn', "couldn't", 't', 'should', 'of', 'few', 'what', 'wasn', 'between', 'before', 'hasn', 'this', 'below', 'than', 'yours', 'wouldn', 'a', 'once', "doesn't", 'isn', 'they', 'himself', 'both', 'then', "aren't", 'we', 'am', 'while', 'shan', 'our', 'there', 'been', 'each', 'until', 's', 'your', 'about', 'why', 'can', "you'll", 'o', 'them', 'out', 'by', "wasn't", 'itself', 'll', 'and', 'm', 'won', 'themselves', 'for', 'me', 'it', 'him', 'how', 'having', 'mightn', 'my', 'does', 'just', 'weren', 'has', "you'd", 'under', 'no', 'aren', 'ain', 'on', 'all', 'who', 'ma', 'in', 'y', 'his', 'did', 'most', 'only', 'these', 'i', 'be', "she's", 'hers', "hasn't", "mightn't", "needn't", 'from', "should've", 'yourselves', 'd', 'mustn', 'some', 'too', 'couldn', 'through', "shan't", 'was', 'which', 'when', 'where', 'doing', 'because', "haven't", 'again', 'whom', "didn't", 'were', 'ours', 'above', 'he',

In [82]:
i = 0
str1 = ' '
final_string = []
all_pos_words = []
all_neg_words = []
s=''

for sent in final.Text.values:
    filterd_sent = []
    sent = cleanHtml(sent)
    for w in sent.split():
        for clean_word in cleanPunc(w).split():
            if((clean_word.isalpha()) & (len(clean_word) > 2)):
                if(clean_word.islower() not in stop):
                    s = (sno.stem(clean_word.lower())).encode('utf8')
                    filterd_sent.append(s)
                    if(final.Score.values)[i] == 'Positive':
                        all_pos_words.append(s)
                    if(final.Score.values)[i] == 'Negative':
                        all_neg_words.append(s)
                else:
                    continue
            else:
                continue
    str1 = b" ".join(filterd_sent)
    final_string.append(str1)
    i += 1

In [83]:
final["CleandText"] = final_string


In [84]:
conn = sql.connect("final.sqlite")
c = conn.cursor()
conn.text_factory = str
final.to_sql('Reviews', conn, schema=None, if_exists='replace')

# Bi-Grams and n-Grams

In [85]:
freq_dist_positive = nltk.FreqDist(all_pos_words)
freq_dist_negative = nltk.FreqDist(all_neg_words)
print("Common Postive words", freq_dist_positive.most_common(20))
print("common negative words", freq_dist_negative.most_common(20))

Common Postive words [(b'the', 940048), (b'and', 694890), (b'this', 354609), (b'for', 296172), (b'that', 212057), (b'have', 193685), (b'with', 192845), (b'you', 180798), (b'but', 177024), (b'are', 165544), (b'was', 146722), (b'not', 145518), (b'they', 141255), (b'like', 138531), (b'tast', 126159), (b'these', 116660), (b'good', 107583), (b'love', 106314), (b'flavor', 106287), (b'them', 104833)]
common negative words [(b'the', 221082), (b'and', 121243), (b'this', 77923), (b'not', 53823), (b'was', 51742), (b'that', 50651), (b'for', 49403), (b'but', 42524), (b'have', 35900), (b'tast', 33876), (b'with', 33520), (b'like', 32136), (b'they', 31585), (b'you', 30445), (b'product', 27341), (b'are', 26609), (b'these', 20647), (b'one', 20203), (b'had', 19260), (b'from', 19242)]


**Observation**: from the above it can be seen that the most common positive and the negative words\
overlabs for eg. 'like' could be used as 'not like' etc..\
so, it is good idea to consider pairs of consequent words (bi-grams) or q sequence of n consecutive words (n-grams)

In [86]:
count_vect = CountVectorizer(ngram_range=(1,2))
final_bigram_counts = count_vect.fit_transform(final.Text.values)

In [88]:
final_bigram_counts.get_shape()

(364171, 2910192)

# TF-IDF

In [89]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(final.Text.values)

In [90]:
final_tf_idf.get_shape()

(364171, 2910192)

In [91]:
featuers = tf_idf_vect.get_feature_names()
len(featuers)

C:\Users\Farahi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


2910192

In [95]:
featuers[100000:1000010]

['ales until',
 'ales ve',
 'ales would',
 'ales you',
 'alessandra',
 'alessandra ambrosia',
 'alessi',
 'alessi added',
 'alessi also',
 'alessi and',
 'alessi are',
 'alessi at',
 'alessi brand',
 'alessi breadsticks',
 'alessi caffe',
 'alessi cento',
 'alessi chicken',
 'alessi coarse',
 'alessi coffees',
 'alessi decaf',
 'alessi decafeinated',
 'alessi decaffenated',
 'alessi dipping',
 'alessi disposable',
 'alessi espresso',
 'alessi fig',
 'alessi fine',
 'alessi for',
 'alessi from',
 'alessi garlic',
 'alessi grinder',
 'alessi has',
 'alessi imported',
 'alessi in',
 'alessi is',
 'alessi lavazza',
 'alessi made',
 'alessi makes',
 'alessi mushroom',
 'alessi or',
 'alessi other',
 'alessi packet',
 'alessi products',
 'alessi richard',
 'alessi rosemary',
 'alessi salt',
 'alessi savoiadi',
 'alessi seems',
 'alessi sell',
 'alessi sesame',
 'alessi soup',
 'alessi soups',
 'alessi split',
 'alessi star',
 'alessi thin',
 'alessi vigo',
 'alessi wasn',
 'alessi went',
 'a

In [96]:
print(final_tf_idf[3,:].toarray()[0])

[0. 0. 0. ... 0. 0. 0.]
